In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

In [ ]:
train_data = pd.read_csv('train_data.csv',sep='\t')
train_data

In [ ]:
train_data['qword']

In [ ]:
def editDistance(s1, s2):
    if len(s1) > len(s2):
        s1, s2 = s2, s1

    distances = range(len(s1) + 1)
    for i2, c2 in enumerate(s2):
        distances_ = [i2+1]
        for i1, c1 in enumerate(s1):
            if c1 == c2:
                distances_.append(distances[i1])
            else:
                distances_.append(1 + min((distances[i1], distances[i1 + 1], distances_[-1])))
        distances = distances_
    return distances[-1]

In [ ]:
def convert_to_introw(row):
    row_list = [0 if x == '' else x for x in row.split('/')]
    return map(int, row_list)

In [ ]:
train_data['qword'] = train_data.apply(lambda row: convert_to_introw(row['qword']), axis = 1)

In [ ]:
# train_data['qword_ed'] = train_data.apply(lambda row: editDistance(std, row['qword']), axis = 1)

In [ ]:
train_data['u_lable_int'] = train_data.apply(lambda row: map(int,row['u_label'].split('/')), axis = 1)

In [ ]:
# train_data['u_lable_int'] = train_data.apply(lambda row: int(np.mean(np.array(row['u_lable_int']))), axis = 1)

In [ ]:
train_data['u_lable_int'].value_counts()

In [ ]:
s = train_data.apply(lambda x: pd.Series(x['u_lable_int']),axis=1).stack().reset_index(level=1, drop=True)

In [ ]:
s.name = 'u_label_stack'

In [ ]:
train_data = train_data.join(s)

In [ ]:
label_count_rsl = train_data['u_label_stack'].value_counts()

In [ ]:
label_list = label_count_rsl.index.tolist()

In [ ]:
label_list = map(int, label_list[0:14])


In [ ]:
for label in label_list:
    train_data[str(label)] = train_data['u_label_stack'].apply(lambda x: +1 if x == label else 0)

In [ ]:
qtag_list = train_data['qtag'].value_counts().index.tolist()

In [ ]:
train_data['qtag'][0] == qtag_list[1]

In [ ]:
for qtag in qtag_list:
    name = 'q' + str(qtag)
    train_data[name] = train_data['qtag'].apply(lambda x: + 1 if x == qtag else 0)

In [ ]:
feature_list = []
for label in label_list:
    feature_list.append(str(label))

In [ ]:
for qtag in qtag_list:
    feature_list.append('q'+str(qtag))

In [ ]:
len(feature_list)

In [ ]:
feature_list = feature_list + ['num_like','num_answear','num_HQanswear']

In [ ]:
def get_Mean_Std(feature_col):
    mean = np.mean(np.array(feature_col))
    std_square = np.sum(np.square(feature_col - mean))/(len(feature_col) - 1)
    std = np.sqrt(std_square)
    return mean, std

In [ ]:
mean_numlike, std_numlike = get_Mean_Std(train_data['num_like'])

In [ ]:
mean_ans, std_ans = get_Mean_Std(train_data['num_answear'])

In [ ]:
mean_hqans, std_hqans = get_Mean_Std(train_data['num_HQanswear'])

In [ ]:
train_data['num_like'] = train_data['num_like'].apply(lambda x: (x - mean_numlike)/std_numlike)

In [ ]:
train_data['num_answear'] = train_data['num_answear'].apply(lambda x: (x - mean_ans)/std_ans)

In [ ]:
train_data['num_HQanswear'] = train_data['num_HQanswear'].apply(lambda x : (x - mean_hqans)/std_hqans)

In [ ]:
train_data[feature_list[0:-3]].groupby(train_data.index).sum()

In [ ]:
train_data[feature_list].to_csv('train_feature_AG.txt')